# GPT-3
* Notebook: [gpt3.ipynb](gpt3.ipynb)
* Purpose: Access OpenAI API to receive model generated answers for prompted questions from SQuAD 1.1
* Link to Paper: [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf)
* Adapted code: [Stanford's CS224U OpenQA Homework](https://github.com/cgpotts/cs224u/blob/master/hw_openqa.ipynb)

**Paper Abstract**:<br/>
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training
on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic
in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of
thousands of examples. By contrast, humans can generally perform a new language task from only
a few examples or from simple instructions – something which current NLP systems still largely
struggle to do. Here we show that scaling up language models greatly improves task-agnostic,
few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art finetuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion
parameters, 10x more than any previous non-sparse language model, and test its performance in
the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or fine-tuning,
with tasks and few-shot demonstrations specified purely via text interaction with the model. GPT-3
achieves strong performance on many NLP datasets, including translation, question-answering, and
cloze tasks, as well as several tasks that require on-the-fly reasoning or domain adaptation, such as
unscrambling words, using a novel word in a sentence, or performing 3-digit arithmetic. At the same
time, we also identify some datasets where GPT-3’s few-shot learning still struggles, as well as some
datasets where GPT-3 faces methodological issues related to training on large web corpora. Finally,
we find that GPT-3 can generate samples of news articles which human evaluators have difficulty
distinguishing from articles written by humans. We discuss broader societal impacts of this finding
and of GPT-3 in general.<br/>

![unsupervised_learning.png](./images/gpt3/unsupervised_learning.png)
**Source**: OpenAI paper, page 7

![in_context_learning.png](./images/gpt3/in_context_learning.png)
**Source**: OpenAI paper, page 3

## Install Dependencies and Set up SQuAD Data

### Install Dependencies

In [ ]:
!pip install torch==1.10.0
!pip install ujson
!pip install transformers
!pip install datasets
!pip install spacy
!pip install gitpython

### SQuAD Setup

In [ ]:
import collections
from collections import namedtuple
from datasets import load_dataset
import numpy as np
import random
import re
import string
from typing import List

In [ ]:
squad = load_dataset("squad")
SquadExample = namedtuple("SquadExample",  "id title context question answers")

In [ ]:
def get_squad_split(squad, split="validation"):
    """
    Use `split='train'` for the train split.
    
    Returns
    -------
    list of SquadExample named tuples with attributes
    id, title, context, question, answers
    
    """    
    fields = squad[split].features
    data = zip(*[squad[split][field] for field in fields])
    return [SquadExample(eid, title, context, question, answers["text"]) 
            for eid, title, context, question, answers in data]

In [ ]:
squad_train = get_squad_split(squad, split="train")
squad_dev = get_squad_split(squad)

## Evaluation Functions
Reused from CS224U's HW_OpenQA (Github repo linked above)

In [ ]:
def _find_generated_answer(tokens, newline="\n" ): 
    """Our LMs tend to insert initial newline characters before
    they begin generating text. This function ensures that we 
    properly capture the true first line as the answer while
    also ensuring that token probabilities are aligned."""        
    answer_token_indices = []
    char_seen = False            
    for i, tok in enumerate(tokens):
        # This is the main condition: a newline that isn't an initial
        # string of newlines:
        if tok == newline and char_seen:
            break
        # Keep the initial newlines for consistency:
        elif tok == newline and not char_seen:
            answer_token_indices.append(i)
        # Proper tokens:
        elif tok != newline:
            char_seen = True
            answer_token_indices.append(i)
    return answer_token_indices 

In [ ]:
def normalize_answer(s: str) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s: str) -> List[str]:
    """Normalize string and split string into tokens."""
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold: str, a_pred: str) -> int:
    """Compute the Exact Match score."""
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1_from_tokens(gold_toks: List[str], pred_toks: List[str]) -> float:
    """Compute the F1 score from tokenized gold answer and prediction."""
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def compute_f1(a_gold: str, a_pred: str) -> float:
    """Compute the F1 score."""
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    return compute_f1_from_tokens(gold_toks, pred_toks)

In [ ]:
def evaluate(examples, prompts, gens):
    """Generic evalution function.
    
    Parameters
    ----------
    examples: iterable of `SquadExample` instances
    prompts: list of str
    preds: list of LM-generated texts to evaluate as answers
    
    Returns
    -------
    dict with keys "em_per", "macro_f1", "examples", where
    each "examples" value is a dict
    
    """        
    results = []
    for ex, prompt, gen in zip(examples, prompts, gens):
        answers = ex.answers
        pred = gen['generated_answer']
        # The result is the highest EM from the available answer strings:
        em = max([compute_exact(ans, pred) for ans in answers])
        f1 = max([compute_f1(ans, pred) for ans in answers])
        gen.update({
            "id": ex.id, 
            "question": ex.question, 
            "prediction": pred, 
            "answers": answers, 
            "em": em,
            "f1": f1
        })
        results.append(gen)
    data = {}        
    data["macro_f1"] = np.mean([d['f1'] for d in results])
    data["em_per"] = sum([d['em'] for d in results]) / len(results)
    data["examples"] = results
    return data

In [ ]:
ex = namedtuple("SquadExample",  "id title context question answers")

examples = [
    ex("0", "CS224u", 
       "The course to take is NLU!", 
       "What is the course to take?", 
       ["NLU", "CS224u"])]

prompts = ["Dear model, Please answer this question!\n\nQ: What is the course to take?\n\nA:"]

gens = [{"generated_answer": "NLU", "generated_text": "NLU\nWho am I?"}]

evaluate(examples, prompts, gens)

## Set Up OpenAI API Access

To use the GPT-3 function, sign up for an OpenAI account at https://beta.openai.com/signup<br/><br/>
That should give you $18 in free credits.<br/><br/>
Once your account is set up, you can get your API key from your account dashboard and paste it in below as the value of `openai_key`.<br/><br/>
Instructions from HW_OpenQA

In [ ]:
openai_key = "PUT YOUR OPENAI KEY HERE"

## Language Model Setup
To use the GPT-3 API, install the OpenAI library:

In [ ]:
!pip install openai

In [ ]:
import openai
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
def run_gpt3(prompts, engine="text-curie-001", temperature=0.1, top_p=0.95, **gpt3_kwargs):
    """To use this function, sign up for an OpenAI account at
        
    https://beta.openai.com/signup
    
    That should give you $18 in free credits, which is more than enough
    for this assignment assuming you are careful with testing.
    
    Once your account is set up, you can get your API key from your 
    account dashboard and paste it in below as the value of 
    `openai.api_key`.
    
    Parameters
    ----------
    prompts : iterable of str
    engine : str
        This has to be one of the models whose name begins with "text".
        The "instruct" class of models can't be used, since they seem
        to depend on some kinds of QA-relevant supervision.        
        For options, costs, and other details: 
        https://beta.openai.com/docs/engines/gpt-3                
    temperature : float
        It seems best to set it low for this task!
    top_p : float
        
    For information about values for `gpt3_kwargs`, see
    
    https://beta.openai.com/docs/api-reference/completions
    
    Returns
    -------
    list of dicts   
    
    """
    # Fill this in with the value from your OpenAI account. First
    # verify that your account is set up with a spending limit that
    # you are comfortable with. If you just opened your account,
    # you should have $18 in credit and so won't need to supply any
    # payment information.
    openai.api_key = openai_key
    
    
    assert engine.startswith("text"), \
        "Please use an engine whose name begins with 'text'."
        
    response = openai.Completion.create(
        engine=engine,       
        prompt=prompts,
        temperature=temperature,
        top_p=top_p,
        echo=False,   # This function will not work
        logprobs=1,   # properly if any of these
        n=1,          # are changed!
        **gpt3_kwargs)
    
    # From here, we parse each example to get the values
    # we need:
    data = []
    for ex, prompt in zip(response["choices"], prompts):
        tokens = ex["logprobs"]["tokens"]
        logprobs = ex["logprobs"]["token_logprobs"]        
        probs = list(np.exp(logprobs))
        if "<|endoftext|>" in tokens:
            end_i = tokens.index("<|endoftext|>")
            tokens = tokens[ : end_i]  # This leaves off the "<|endoftext|>"
            probs = probs[ : end_i]    # token -- perhaps dubious.
        ans_indices = _find_generated_answer(tokens)
        answer_tokens = [tokens[i] for i in ans_indices]
        answer_probs = [probs[i] for i in ans_indices]
        answer = "".join(answer_tokens)        
        data.append({
            "prompt": prompt,
            "generated_text": ex["text"],
            "generated_tokens": tokens,
            "generated_probs": probs,
            "generated_answer": answer,
            "generated_answer_tokens": answer_tokens,
            "generated_answer_probs": answer_probs})
        
    return data

In [ ]:
gpt3_ex = run_gpt3([
     "What year was Stanford University founded?",
     "In which year did Stanford first enroll students?"])

gpt3_ex

## Functions to Built and Evaluate Few Shot QA

In [ ]:
def build_few_shot_qa_prompt(ex, squad_train, n_context=2, joiner="\n\n"):
    segs = []
    train_exs = random.sample(squad_train, k=n_context)    
    for t in train_exs:
        segs += [
            f"Title: {t.title}",
            f"Background: {t.context}",
            f"Q: {t.question}",
            f"A: {t.answers[0]}"
        ]
    segs += [
        f"Title: {ex.title}",
        f"Background: {ex.context}",
        f"Q: {ex.question}",
        f"A:"
    ]
    return joiner.join(segs)

In [ ]:
print(build_few_shot_qa_prompt(squad_dev[0], squad_train, n_context=1))

In [ ]:
import time

def evaluate_few_shot_qa(examples, squad_train, gen_func=run_gpt3, batch_size=20, n_context=2):
    prompts = []
    gens = []
    for i in range(0, len(examples), batch_size):
        batch = examples[i: i+batch_size]
        ps = [build_few_shot_qa_prompt(ex, squad_train, n_context=n_context) for ex in batch]        
        gs = gen_func(ps)       
        prompts += ps
        gens += gs
        time.sleep(10) # GPT rate limit exceeded if more than 60 requests received in a minute
    return evaluate(examples, prompts, gens)

## Evaluate SQuAD Dev Set Using GPT-3

### NOTE: Running These Cells Will Charge You around 9 Dollars in OpenAI

In [ ]:
%%time
few_shot_qa_results_gpt3 = evaluate_few_shot_qa(
    squad_dev, squad_train, n_context=1, gen_func=run_gpt3)

print(few_shot_qa_results_gpt3['macro_f1'])

In [ ]:
print("Macro F1: " + str(few_shot_qa_results_gpt3['macro_f1']))
print("Exact Match: " + str(few_shot_qa_results_gpt3["em_per"]))